In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROA_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_groq import ChatGroq
model =ChatGroq(model="llama-3.3-70b-versatile",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10acba9b0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10acbbe20>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
## without prompt templates

from langchain_core.messages import HumanMessage, AIMessage
model.invoke(
[
    HumanMessage(content="hi i am ujjwal sinha data science learner and blockchain developer and student who can teach"),
    AIMessage(content="hello  ujjwal ")
    
])

AIMessage(content=' nice to meet you. as a data science learner and blockchain developer, you have a unique combination of skills. data science and blockchain are both exciting fields with a lot of potential for innovation and growth.\n\nwhat specific areas of data science are you interested in (e.g. machine learning, natural language processing, computer vision). and in blockchain, are you more focused on development (e.g. smart contracts, dapps) or other aspects like cryptography or blockchain architecture.\n\nalso, what kind of teaching are you interested in. are you looking to create online courses, teach in a classroom setting, or mentor students one-on-one. let me know and i can try to help you get started.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 139, 'prompt_tokens': 60, 'total_tokens': 199, 'completion_time': 0.505454545, 'prompt_time': 0.002993572, 'queue_time': 0.067133992, 'total_time': 0.508448117}, 'model_name': 'llama-3.3-70b-versat

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!


In [4]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
        return store[session_id]
    
with_runnable_history=RunnableWithMessageHistory(model,get_session_history)


In [5]:
config={"configurable":{"session_id":"chat1"}}

In [6]:
response=with_runnable_history.invoke(
    [HumanMessage(content="hi my name is ujjwal sinha , I am a data scientist and blockchain developer ")],
    config=config,
    )

response.content

"Nice to meet you, Ujjwal Sinha. It's great to connect with someone who has expertise in both data science and blockchain development. Those are two fascinating fields that are increasingly intersecting, especially with the rise of decentralized data management and analytics.\n\nAs a data scientist, I'm sure you're familiar with the power of data-driven insights and the importance of working with complex datasets. And as a blockchain developer, you must be excited about the potential of blockchain technology to create secure, transparent, and decentralized systems.\n\nWhat kind of projects have you been working on recently? Are you building any blockchain-based applications or analyzing data for any interesting use cases? I'd love to hear more about your work and experiences."

In [7]:
#prompt Template
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt|model


In [8]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Ujjwal sinha")]})

AIMessage(content="Nice to meet you, Ujjwal Sinha. It's a pleasure to assist you. How can I help you today? Do you have any questions or topics you'd like to discuss? I'm all ears.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 60, 'total_tokens': 105, 'completion_time': 0.163636364, 'prompt_time': 0.009830062, 'queue_time': 0.058949218, 'total_time': 0.173466426}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_72a5dc99ee', 'finish_reason': 'stop', 'logprobs': None}, id='run-89015cfc-481c-49e8-aea3-f0cfb2350bd8-0', usage_metadata={'input_tokens': 60, 'output_tokens': 45, 'total_tokens': 105})

In [9]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [10]:
config1 = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Ujjwal")],
    config=config1
)

response

AIMessage(content="Hello Ujjwal, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 58, 'total_tokens': 86, 'completion_time': 0.101818182, 'prompt_time': 0.003276496, 'queue_time': 0.055404159, 'total_time': 0.105094678}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9a8b91ba77', 'finish_reason': 'stop', 'logprobs': None}, id='run-e73c4389-2e54-45c5-aa95-46f87ff6ae3a-0', usage_metadata={'input_tokens': 58, 'output_tokens': 28, 'total_tokens': 86})

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config1,
)

response.content

AttributeError: 'NoneType' object has no attribute 'messages'

In [19]:
#managing history 

from langchain_core.messages import SystemMessage, trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

/Users/ujjwalsinha/Desktop/langchain /venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [20]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain =( RunnablePassthrough(messages=itemgetter("messages")| trimmer)
        |prompt|model
        
        
        
        
        
        )

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content


'You asked what 2 + 2 is, and I told you the answer is 4.'